<a href="https://colab.research.google.com/github/krakowiakpawel9/decision_tree_course/blob/master/05_decision_tree_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

# https://medium.com/@rakendd/decision-tree-from-scratch-9e23bcfb4928

In [0]:
# najmniejsza reprezentowalna liczba
epsilon = np.finfo(float).eps

In [0]:
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [0]:
df = pd.DataFrame(dataset)
df

,Taste,Temperature,Texture,Eat
0,Salty,Hot,Soft,No
1,Spicy,Hot,Soft,No
2,Spicy,Hot,Hard,Yes
3,Spicy,Cold,Hard,No
4,Spicy,Hot,Hard,Yes
5,Sweet,Cold,Soft,Yes
6,Salty,Cold,Soft,No
7,Sweet,Hot,Soft,Yes
8,Spicy,Cold,Soft,Yes
9,Salty,Hot,Hard,Yes


### Entropia:
### $$E = - \sum_{i=1}^{N}p_{i} \cdot log_{2}p_{i}$$

gdzie $p_{i}$ jest proporcją  liczby elementów w tej grupie podziału do liczby elementów w grupie przed podziałem

In [0]:
entropy_node = 0
values = df.Eat.unique()

for value in values:
    fraction = df.Eat.value_counts()[value] / len(df.Eat)
    entropy_node += -fraction * np.log2(fraction)
    
entropy_node    

0.9709505944546686

In [0]:
def calc_entropy(attribute):
    target_variables = df.Eat.unique()
    variables = df[attribute].unique()
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df.Eat == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy_each_feature += -fraction * np.log2(fraction + epsilon)
        fraction2 = den / len(df)
        entropy_attribute += -fraction2 * entropy_each_feature
    entropy_attribute = abs(entropy_attribute)
    return entropy_attribute

In [0]:
def information_gain(entropy_node, entropy_attribute):
    return entropy_node - entropy_attribute

In [0]:
entropy_Taste = calc_entropy('Taste')
entropy_Temp = calc_entropy('Temperature')
entropy_Texture = calc_entropy('Texture')

In [0]:
ig_Taste = information_gain(entropy_node, entropy_Taste)
ig_Temp = information_gain(entropy_node, entropy_Temp)
ig_Texture = information_gain(entropy_node, entropy_Texture)

print('Zysk informacyjny {}: {}'.format('Taste', round(ig_Taste, 4)))
print('Zysk informacyjny {}: {}'.format('Temperature', round(ig_Temp, 4)))
print('Zysk informacyjny {}: {}'.format('Texture', round(ig_Texture, 4)))

Zysk informacyjny Taste: 0.21
Zysk informacyjny Temperature: 0.02
Zysk informacyjny Texture: 0.0464


In [0]:
def find_entropy(df):
    label = df.keys()[-1]
    entropy = 0
    values = df[label].unique()
    for value in values:
        fraction = df[label].value_counts()[value] / len(df[label])
        entropy += -fraction * np.log2(fraction)
    return entropy

def find_entropy_attribute(df, attribute):
    label = df.keys()[-1]
    target_variables = df[label].unique()
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df[label] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy += - fraction * np.log2(fraction + epsilon)
        fraction2 = den / len(df)
        entropy2 += - fraction2 * entropy
    return abs(entropy2)
    
    
def find_winner(df):
    entropy_attr = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def build_tree(df, tree=None):
    
    label = df.keys()[-1]
    
    node = find_winner(df)
    
    att_values = np.unique(df[node])
    
    if tree is None:
        tree = {}
        tree[node] = {}
        
    for value in att_values:
        
        subtable = get_subtable(df, node, value)
        cl_value, counts = np.unique(subtable['Eat'], return_counts=True)
        
        if len(counts) == 1:
            tree[node][value] = cl_value[0]
        else:
            tree[node][value] = build_tree(subtable)
            
    return tree

In [0]:
tree = build_tree(df)
tree

{'Taste': {'Salty': {'Texture': {'Hard': 'Yes', 'Soft': 'No'}},
  'Spicy': {'Temperature': {'Cold': {'Texture': {'Hard': 'No', 'Soft': 'Yes'}},
    'Hot': {'Texture': {'Hard': 'Yes', 'Soft': 'No'}}}},
  'Sweet': 'Yes'}}

In [0]:
import pprint
pprint.pprint(tree)

{'Taste': {'Salty': {'Texture': {'Hard': 'Yes', 'Soft': 'No'}},
           'Spicy': {'Temperature': {'Cold': {'Texture': {'Hard': 'No',
                                                          'Soft': 'Yes'}},
                                     'Hot': {'Texture': {'Hard': 'Yes',
                                                         'Soft': 'No'}}}},
           'Sweet': 'Yes'}}


In [0]:
def predict(inst, tree):
    
    for nodes in tree.keys():
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break
    return prediction

In [0]:
inst = df.iloc[6]
inst

Taste          Salty
Temperature     Cold
Texture         Soft
Eat               No
Name: 6, dtype: object

In [0]:
predict(inst, tree)

'No'